In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.optimize import minimize

In [2]:
# Load data
data = pd.read_csv('../data/processed/combined_data.csv', parse_dates=['Date'])
data

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Daily_Return,Rolling_Mean,Rolling_Std
0,2015-01-02 00:00:00+00:00,-1.078566,-1.127493,-1.127500,-1.128401,-1.125787,0.046547,TSLA,NaN,NaN,NaN
1,2015-01-05 00:00:00+00:00,-1.083006,-1.131838,-1.130658,-1.131299,-1.129708,0.179019,TSLA,-1.966623,NaN,NaN
2,2015-01-06 00:00:00+00:00,-1.082433,-1.131277,-1.131734,-1.132700,-1.131823,0.374930,TSLA,0.226061,NaN,NaN
3,2015-01-07 00:00:00+00:00,-1.082592,-1.131432,-1.131463,-1.130054,-1.130273,-0.347294,TSLA,-0.106076,NaN,NaN
4,2015-01-08 00:00:00+00:00,-1.082751,-1.131588,-1.131921,-1.129945,-1.130527,-0.243330,TSLA,-0.106185,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7417,2024-10-24 00:00:00+00:00,3.000153,2.863617,2.838327,2.879085,2.868996,-0.486850,SPY,0.065118,2.844483,0.224719
7418,2024-10-25 00:00:00+00:00,2.998708,2.862204,2.869202,2.889846,2.879812,-0.307205,SPY,-0.050156,2.848523,0.186923
7419,2024-10-28 00:00:00+00:00,3.011639,2.874857,2.856922,2.907234,2.887376,-0.557099,SPY,0.107805,2.852790,0.154502
7420,2024-10-29 00:00:00+00:00,3.018429,2.881501,2.858325,2.892340,2.868077,-0.371069,SPY,0.040101,2.857226,0.119094


In [3]:
import pandas as pd

# Select only numeric columns for return calculation
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# Calculate daily returns for numeric columns only, dropping NaNs created by pct_change
daily_returns = numeric_data.pct_change().dropna()

# Calculate annualized average return (assuming 252 trading days in a year)
annual_returns = daily_returns.mean() * 252

# Calculate the annualized covariance matrix of returns
cov_matrix = daily_returns.cov() * 252

C:\Users\1221\AppData\Local\Temp\ipykernel_5160\1991545478.py:7: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = numeric_data.pct_change().dropna()


### Annualized Average Returns

In [4]:
annual_returns

Adj Close          5.830529
Close             -2.608971
High              -6.213796
Low               -1.440908
Open               1.792204
Volume           179.675747
Daily_Return   -1397.491905
Rolling_Mean       0.090642
Rolling_Std        0.021920
dtype: float64

### Annualized Covariance Matrix of Returns

In [5]:
cov_matrix

,Adj Close,Close,High,Low,Open,Volume,Daily_Return,Rolling_Mean,Rolling_Std
Adj Close,3451.738067,-2.385899,-0.570532,-0.987568,-0.293281,-17.171367,1.841485e+01,0.130321,0.336709
Close,-2.385899,203.404297,133.556180,2.712356,11.982528,-11.052597,-4.799674e-01,0.134777,-0.127404
High,-0.570532,133.556180,272.278940,-5.720196,16.379385,16.808356,-3.117347e+01,0.577222,-0.842462
Low,-0.987568,2.712356,-5.720196,915.543940,75.028193,-10.824726,6.804187e+00,-0.303920,-9.601580
Open,-0.293281,11.982528,16.379385,75.028193,406.226654,-9.557410,-1.746567e+01,-0.248499,-1.189846
Volume,-17.171367,-11.052597,16.808356,-10.824726,-9.557410,704245.219520,-4.020069e+03,1.472757,14.045591
Daily_Return,18.414850,-0.479967,-31.173473,6.804187,-17.465672,-4020.068666,3.220707e+07,0.668641,8.907388
Rolling_Mean,0.130321,0.134777,0.577222,-0.303920,-0.248499,1.472757,6.686406e-01,10.328177,0.370316
Rolling_Std,0.336709,-0.127404,-0.842462,-9.601580,-1.189846,14.045591,8.907388e+00,0.370316,443.511147


### Define Portfolio Weights and Metrics

In [6]:
import numpy as np

# Calculate equal allocation weights based on the number of assets
num_assets = len(annual_returns)
weights = np.array([1 / num_assets] * num_assets)

# Portfolio metrics calculation
def portfolio_performance(weights, annual_returns, cov_matrix, risk_free_rate=0.02):
    portfolio_return = np.dot(weights, annual_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    return portfolio_return, portfolio_volatility, sharpe_ratio

# Calculate and print the initial portfolio metrics
portfolio_return, portfolio_volatility, sharpe_ratio = portfolio_performance(weights, annual_returns, cov_matrix)
print(f"Initial Portfolio Return: {portfolio_return:.2%}")
print(f"Initial Portfolio Volatility: {portfolio_volatility:.2%}")
print(f"Initial Sharpe Ratio: {sharpe_ratio:.2f}")

Initial Portfolio Return: -13559.38%
Initial Portfolio Volatility: 63740.75%
Initial Sharpe Ratio: -0.21


### Optimize Portfolio Weights to Maximize the Sharpe Ratio

In [7]:
# Function to minimize (negative Sharpe Ratio)
def neg_sharpe(weights, annual_returns, cov_matrix, risk_free_rate=0.02):
    return -portfolio_performance(weights, annual_returns, cov_matrix, risk_free_rate)[2]

# Constraints and bounds for the optimization
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Weights sum to 1
bounds = tuple((0, 1) for _ in range(len(weights)))  # Each weight is between 0 and 1

# Optimization
opt_results = minimize(neg_sharpe, weights, args=(annual_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights and performance
optimal_weights = opt_results.x
optimal_return, optimal_volatility, optimal_sharpe = portfolio_performance(optimal_weights, annual_returns, cov_matrix)
print("Optimal Weights:", optimal_weights)
print(f"Optimal Portfolio Return: {optimal_return:.2f}")
print(f"Optimal Portfolio Volatility: {optimal_volatility:.2f}")
print(f"Optimal Sharpe Ratio: {optimal_sharpe:.2f}")

Optimal Weights: [1.27811066e-01 5.69847826e-08 1.94471966e-09 1.21020891e-07
 3.32277538e-01 1.93544801e-02 4.76913502e-09 5.19930045e-01
 6.27215488e-04]
Optimal Portfolio Return: 4.87
Optimal Portfolio Volatility: 19.17
Optimal Sharpe Ratio: 0.25


In [9]:
# Assuming daily_returns contains returns for Tesla, we proceed with 'Daily_Return' column
tsla_returns = daily_returns['Daily_Return']

# Calculate Value at Risk (VaR) at 95% confidence level
confidence_level = 0.95
VaR_tsla = tsla_returns.quantile(1 - confidence_level)
print(f"Value at Risk (95% Confidence) for Tesla: {VaR_tsla:.4f}")

Value at Risk (95% Confidence) for Tesla: -8.1220
